In [1]:
%%capture

!pip install gdown gpxpy

In [17]:
%%capture

import gdown
DRIVE_FOLDER = 'https://drive.google.com/drive/folders/1SFjNDTuglgnWmVSY-I47xiekc-iqMCG_?usp=sharing'
gdown.download_folder(DRIVE_FOLDER, quiet=True)

In [7]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

osmtracker_df = parse_gpx('/content/multipath/osm-upload7163940511671199185.gpx')
osmtracker_df.head()

,time,lat,lon
0,2024-11-26 14:32:47+00:00,20.993706,105.868735
1,2024-11-26 14:32:48+00:00,20.993707,105.868711
2,2024-11-26 14:32:49+00:00,20.993702,105.868678
3,2024-11-26 14:32:50+00:00,20.993695,105.868643
4,2024-11-26 14:32:51+00:00,20.993696,105.868611


In [19]:
constellation_names = {
    0: 'Unknown',
    1: 'GPS',
    3: 'Glonass',
    4: 'QZSS',
    5: 'Beidou',
    6: 'Galileo',
}

gnss_logger_df = pd.read_csv('/content/multipath/raw.csv')

gnss_logger_df['time'] = pd.to_datetime(gnss_logger_df['utcTimeMillis'], unit='ms', utc=True)
gnss_logger_df['ConstellationName'] = gnss_logger_df['ConstellationType'].map(constellation_names).astype(str)

gnss_logger_df.head()

,utcTimeMillis,Svid,ConstellationType,MultipathIndicator,time,ConstellationName
0,1732631567000,3,1,2,2024-11-26 14:32:47+00:00,GPS
1,1732631567000,6,1,1,2024-11-26 14:32:47+00:00,GPS
2,1732631567000,11,1,0,2024-11-26 14:32:47+00:00,GPS
3,1732631567000,14,1,2,2024-11-26 14:32:47+00:00,GPS
4,1732631567000,17,1,1,2024-11-26 14:32:47+00:00,GPS


In [23]:
gnss_logger_df['ConstellationName'].value_counts()

,count
ConstellationName,
Beidou,28726
GPS,19174
Glonass,12986
Galileo,11525
QZSS,7128


In [21]:
df = pd.merge(osmtracker_df, gnss_logger_df, on='time', how='inner')
df.head()

,time,lat,lon,utcTimeMillis,Svid,ConstellationType,MultipathIndicator,ConstellationName
0,2024-11-26 14:32:47+00:00,20.993706,105.868735,1732631567000,3,1,2,GPS
1,2024-11-26 14:32:47+00:00,20.993706,105.868735,1732631567000,6,1,1,GPS
2,2024-11-26 14:32:47+00:00,20.993706,105.868735,1732631567000,11,1,0,GPS
3,2024-11-26 14:32:47+00:00,20.993706,105.868735,1732631567000,14,1,2,GPS
4,2024-11-26 14:32:47+00:00,20.993706,105.868735,1732631567000,17,1,1,GPS


In [26]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'QZSS']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

In [27]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'Galileo']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

In [28]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'Glonass']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.